# Event data

Scipp can handle *event data*, a certain type of sparse data, i.e., data that cannot directly be represented as a multi-dimensional array.
For applications that rely solely on dense arrays of data this section can safely be ignored.

Scipp supports event data in shape of a multi-dimensional array of lists.
This could, e.g., be used to store data from an array of sensors/detectors that are read out independently, with potentially widely varying frequency.

Event data can be created using a special `dtype`, `event_list_float64`, `event_list_float32`, `event_list_int64`, and `event_list_int32`.
Since here we are not dealing with a dense array we cannot set values for all `x` from a numpy array.
The recommended approach is to slice out all dimensions.
Then the remaining values (for a particluar "x" in this case) are a dense array with a list-like interface.
Initially all lists are empty:

In [ ]:
import numpy as np
import scipp as sc

var = sc.Variable(dims=['x'],
                  shape=[4],
                  dtype=sc.dtype.event_list_float64)
sc.show(var)
var

In [ ]:
var['x', 0].values = np.arange(3)
var['x', 1].values.append(42)
var['x', 0].values.extend(np.ones(3))
var['x', 3].values = np.ones(6)
sc.show(var)
var

In [ ]:
var['x', 0].values

In [ ]:
var['x', 1].values

In [ ]:
var['x', 2].values

Operations between variables or datasets broadcast dense data to lists:

In [ ]:
scale = sc.Variable(dims=['x'], values=np.arange(2.0, 6))
var *= scale
var['x', 0].values

In [ ]:
var['x', 1].values

In [ ]:
var['x', 2].values

Event data in a data array can be associated with a corresponding event coordinate and event labels:

In [ ]:
a = sc.DataArray(data=var, coords={
    'x': sc.Variable(['x'], values=np.arange(4.0)),
    'y': var})
var['x', 0].values = np.arange(7)
var['x', 3].values = np.ones(2)
sc.show(a)
a

The lengths of the sublists between coordinate and values (and variances) must match.
Scipp does not enforce this when modifying sublists, but *does* verify correctness in operations on variables or data arrays.